Из этих данных можно попробовать оценить среднюю продолжительность болезни с помощью [взаимнокорреляционной функции](https://ru.wikipedia.org/wiki/%D0%92%D0%B7%D0%B0%D0%B8%D0%BC%D0%BD%D0%BE%D0%BA%D0%BE%D1%80%D1%80%D0%B5%D0%BB%D1%8F%D1%86%D0%B8%D0%BE%D0%BD%D0%BD%D0%B0%D1%8F_%D1%84%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D1%8F): за среднюю продолжительность болезни возьмём такое смещение графика `confirmed` вправо, что он максимально коррелирует с сумой остальных двух. Для этого посчитаем взаимокорреляционную функцию $f \star g$ величин $f = \text{confirmed}$ и $g = \text{recovered} + \text{deaths}$ и возьмём в качестве смещения 

$$
i^* = \text{argmax}_{i} (f \star g).
$$

Взаимокореляционную функцию (cross corelation) можно вычислить с помощью метода [signal.correlate](https://docs.scipy.org/doc/scipy/reference/reference/generated/scipy.signal.correlate.html#scipy.signal.correlate) из библиотеки `SciPy`.


<details>
<summary>Разница между режимами работы signal.correlate</summary>

Если `mode='valid'`, то функция считает корреляцию для всех таких положений `g`, при которых `f` не надо дополнять нулями по краям.  

```{figure} /_static/lecture_specific/pandas/correlate_valid.svg
```

Если `mode='valid'`, то функция считает корреляцию для всех положений `g`, при которых `f` хотя бы одним элементом накладывается на `g`. 

```{figure} /_static/lecture_specific/pandas/correlate_full.svg
```

Если длины `f` и `g` равны 5 и 3 соответственно, то результирующий массив в режиме `valid` выходит размера 3, а в режиме `full` --- 5. 

</details>

In [ ]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 22})

n = 100
sigma = 0.05

def triangle(center, width, height):
    half_width = width / 2
    k = height / half_width
    
    def left_side(x):
        root = center - half_width
        return k * (x - root) 
    
    def right_side(x):
        root = center + half_width
        return -k * (x - root)
    
    
    x = np.arange(n)
    y = left_side(x) * (x < center) + right_side(x) * (x >= center)
    return y * (y >= 0)


x = np.arange(n)
f = triangle(40, 30, 1) 
g = triangle(30, 30, 0.9)

noisy_f = f + np.random.normal(0, sigma, size=(n, ))
noisy_g = g + np.random.normal(0, sigma, size=(n, ))

corr = signal.correlate(noisy_f, noisy_g, mode="full")
shift = n - np.argmax(corr) - 1


fig, axs = plt.subplots(nrows=4, ncols=1, figsize=(10, 12))        
axs[0].plot(x, f, label="f")
axs[0].plot(x, g, label="g")
axs[0].legend()
axs[1].plot(x, noisy_f, label="f")
axs[1].plot(x, noisy_g, label="g")
axs[1].legend()
axs[2].plot(corr, label="f*g")
axs[2].legend()
axs[3].plot(x + shift, noisy_f, label="f")
axs[3].plot(x, noisy_g, label="g")
axs[3].legend()
plt.show(fig)